<a href="https://colab.research.google.com/github/pipecode-br/deep_learning_pytorch/blob/main/architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Etapa 1: Importar biblioteca

In [9]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
torch.__version__

'1.10.0+cu111'

In [11]:
torch.manual_seed(123)

### Etapa 2: Construir modelo